In [1]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.3 MB/s 


In [2]:
import ee
from unidecode import unidecode
import os


In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=YuJGkh-u2j7z927sfnNAAdY_H1DkTEkRSu_ULUzogcE&tc=TVtO6I3XKMP-dMC7daUpQLdIpRo-FIR-5nPKR1jHZiw&cc=0VMRfUdqggXKOeqOH6Upp-3LbvVti18qA3YgBWI8zdY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqmYIOCHXqsQ3nD8H91OB46AUsIyhVPYaHg6IaJs_EN_jVGXE8T2Z0

Successfully saved authorization token.


In [6]:
gdrive_folder = 'Italy temp data'


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
italy = 'users/basanthqo/Italy_prov'
county_region = ee.FeatureCollection(italy)
feature_list = county_region.toList(1e5)
feature_list_computed = feature_list.getInfo()


In [9]:
CLEAN_NAME = lambda r, l: r.get('properties').get(l)

In [26]:
def ftr_key_fn(region):
 return f"DEN_PROV_{CLEAN_NAME(region, 'DEN_PROV')}-COD_REG_{CLEAN_NAME(region,'COD_REG')}-COD_PROV_{CLEAN_NAME(region, 'COD_PROV')}"


In [22]:
# image collection bands
surf_temp_ems_band = ['LST_Day_1km','LST_Night_1km']


In [23]:
#MODIS aqua land surface temparature and emmisivity
surf_temp_ems = ee.ImageCollection("MODIS/061/MYD11A2")

In [24]:
bands = surf_temp_ems_band

In [18]:
def appendBand(current, previous):
    # Rename the band
    previous=ee.Image(previous)
    current = current.select(bands)
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

In [27]:
def export_to_cloud(img, fname, folder, expregion, eeuser=None, scale=500):
  # print "export to cloud"
  expcoord = expregion.geometry().coordinates().getInfo()[0]
  expconfig = dict(description=fname, folder=folder, fileNamePrefix=fname, dimensions=None, region=expcoord,
                   scale=scale, crs='EPSG:4326', crsTransform=None, maxPixels=1e13)
  task = ee.batch.Export.image.toDrive(image=img.clip(expregion), **expconfig)
  task.start()
  while task.status()['state'] == 'RUNNING':
    print ('Running...')
    time.sleep(10)
  print ('Done.', task.status())

In [29]:
start_date ='2012-01-1'
end_date = '2022-09-30'

In [30]:
imgcoll = ee.ImageCollection(surf_temp_ems).filterDate(start_date,end_date)
img=imgcoll.iterate(appendBand)
img=ee.Image(img)

In [32]:
imgcoll.size().getInfo()

491

In [33]:
keys_with_issues = []
count_already_downloaded = 0
count_filtered = 0

for idx, region in enumerate(feature_list_computed):
  if not True:
      count_filtered += 1
      continue
      
  subunit_key = ftr_key_fn(region)
  file_name = f'Italy_surf_temp_ems_{subunit_key}_{start_date}_{end_date}'
  print(file_name)
  if os.path.isfile(os.path.join(file_name + '.tif')):
          print (subunit_key, 'already downloaded. Continuing...')
          count_already_downloaded += 1
          continue

  try:
      export_to_cloud(img, file_name, gdrive_folder, ee.Feature(region), scale=500)
  except KeyboardInterrupt:
      print ('received SIGINT, terminating execution')
      break
  except Exception as e:
    print ('issue with {} ({})'.format(subunit_key, str(e)))

Italy_surf_temp_ems_DEN_PROV_Vercelli-COD_REG_1-COD_PROV_2_2012-01-1_2022-09-30
Done. {'state': 'READY', 'description': 'Italy_surf_temp_ems_DEN_PROV_Vercelli-COD_REG_1-COD_PROV_2_2012-01-1_2022-09-30', 'creation_timestamp_ms': 1665772939098, 'update_timestamp_ms': 1665772939098, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'YYZKBNFRT4E46ZU6HK6AUZ6U', 'name': 'projects/earthengine-legacy/operations/YYZKBNFRT4E46ZU6HK6AUZ6U'}
Italy_surf_temp_ems_DEN_PROV_Novara-COD_REG_1-COD_PROV_3_2012-01-1_2022-09-30
Done. {'state': 'READY', 'description': 'Italy_surf_temp_ems_DEN_PROV_Novara-COD_REG_1-COD_PROV_3_2012-01-1_2022-09-30', 'creation_timestamp_ms': 1665772943527, 'update_timestamp_ms': 1665772943527, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UKMVVTMGEN6YAJOKASJTLIKZ', 'name': 'projects/earthengine-legacy/operations/UKMVVTMGEN6YAJOKASJTLIKZ'}
Italy_surf_temp_ems_DEN_PROV_Cuneo-COD_REG_1-COD_PROV_4_2012-01-1_2022-09-30
Done. {'state': 'READY', 'description':